### 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Forecasting libraries
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.optimize import curve_fit

# Import custom functions from Task 3
import sys
sys.path.append('../src')
try:
    from event_impact_model import calculate_combined_impact, forecast_with_events
    print("✓ Successfully imported event impact functions")
except ImportError:
    print("⚠️ Custom module not found, defining functions locally")
    
# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
np.random.seed(42)

# %%
# Load data from previous tasks
print("Loading data from previous tasks...")

# Load enriched dataset
df = pd.read_csv('../data/processed/ethiopia_fi_enriched.csv')
observations = df[df['record_type'] == 'observation'].copy()
events = df[df['record_type'] == 'event'].copy()

# Convert dates
observations['observation_date'] = pd.to_datetime(observations['observation_date'], errors='coerce')
events['event_date'] = pd.to_datetime(events['event_date'], errors='coerce')

# Load impact modeling matrices from Task 3
association_matrix = pd.read_csv('../data/processed/event_indicator_association_matrix.csv', index_col=0)
lag_matrix = pd.read_csv('../data/processed/event_impact_lags.csv', index_col=0)
direction_matrix = pd.read_csv('../data/processed/event_impact_directions.csv', index_col=0)
magnitude_matrix = pd.read_csv('../data/processed/event_impact_magnitudes.csv', index_col=0)

# Load key insights and limitations
key_insights = pd.read_csv('../data/processed/key_insights.csv')
limitations = pd.read_csv('../data/processed/impact_modeling_limitations.csv')

print("✓ Data loaded successfully")

### 2. Define Targets and Prepare Data

In [ ]:
print("="*60)
print("DEFINING FORECASTING TARGETS")
print("="*60)

# Primary targets from business objective
primary_targets = {
    'ACC_OWNERSHIP': {
        'name': 'Account Ownership Rate',
        'description': '% of adults with account at financial institution or mobile money',
        'pillar': 'Access',
        'unit': 'percentage',
        'current_value': None,
        'nfis_target': 60  # NFIS-II target by 2027
    },
    'USG_DIGITAL_PAYMENT': {
        'name': 'Digital Payment Adoption Rate',
        'description': '% of adults who made or received digital payment',
        'pillar': 'Usage',
        'unit': 'percentage',
        'current_value': None,
        'nfis_target': None  # No specific NFIS target
    }
}

# Get latest values for targets
for target_code in primary_targets.keys():
    target_data = observations[observations['indicator_code'] == target_code].copy()
    if not target_data.empty:
        latest = target_data.sort_values('observation_date').iloc[-1]
        primary_targets[target_code]['current_value'] = latest['value_numeric']
        primary_targets[target_code]['latest_date'] = latest['observation_date']
        print(f"✓ {primary_targets[target_code]['name']}: {latest['value_numeric']}% (as of {latest['observation_date'].strftime('%Y-%m')})")

# Also prepare data for key enabling indicators
enabling_indicators = ['ACC_MM_ACCOUNT', 'INF_AGENT_DENSITY', 'ENB_MOBILE_INTERNET', 'ENB_SMARTPHONE_PEN']


### 3. Select Forecasting Approach

In [ ]:
print("\n" + "="*60)
print("SELECTING FORECASTING APPROACH")
print("="*60)

# Given sparse data (5 Findex points over 13 years), we consider multiple approaches:
forecasting_approaches = {
    'approach_1': {
        'name': 'Trend Regression',
        'description': 'Linear/Log regression on historical trend',
        'pros': ['Simple', 'Interpretable', 'Works with sparse data'],
        'cons': ['Assumes trend continuation', 'Ignores events'],
        'suitability': 'Medium - Good for baseline'
    },
    'approach_2': {
        'name': 'Event-Augmented Model',
        'description': 'Trend + Event impacts from Task 3',
        'pros': ['Incorporates known events', 'More realistic'],
        'cons': ['Requires impact estimation', 'More complex'],
        'suitability': 'High - Recommended for main forecast'
    },
    'approach_3': {
        'name': 'Scenario Analysis',
        'description': 'Optimistic/Base/Pessimistic scenarios',
        'pros': ['Handles uncertainty', 'Useful for planning'],
        'cons': ['Subjective scenario definitions'],
        'suitability': 'High - Essential given data limitations'
    },
    'approach_4': {
        'name': 'ARIMA/Exponential Smoothing',
        'description': 'Time series models',
        'pros': ['Statistical rigor', 'Confidence intervals'],
        'cons': ['Requires more data points', 'May overfit'],
        'suitability': 'Low - Insufficient data points'
    }
}

print("\n📊 Available Forecasting Approaches:")
for key, approach in forecasting_approaches.items():
    print(f"\n{approach['name']}:")
    print(f"  Description: {approach['description']}")
    print(f"  Pros: {', '.join(approach['pros'][:2])}")
    print(f"  Cons: {', '.join(approach['cons'][:2])}")
    print(f"  Suitability: {approach['suitability']}")

# Decision: Use combination of approaches
print("\n🎯 Selected Approach: Combined Model")
print("  1. Baseline: Trend regression")
print("  2. Main forecast: Event-augmented model")
print("  3. Uncertainty: Scenario analysis with confidence intervals")


### 4. Prepare Time Series Data

In [ ]:
def prepare_time_series(indicator_code, freq='A'):
    """
    Prepare time series data for a given indicator.
    
    Parameters:
    -----------
    indicator_code : str
        Code of the indicator to forecast
    freq : str
        Frequency for resampling ('A' for annual, 'M' for monthly)
    
    Returns:
    --------
    ts : pd.Series
        Time series with datetime index
    metadata : dict
        Additional information about the series
    """
    
    # Filter data for the indicator
    ind_data = observations[observations['indicator_code'] == indicator_code].copy()
    
    if ind_data.empty:
        print(f"⚠️ No data found for {indicator_code}")
        return None, None
    
    # Sort by date
    ind_data = ind_data.sort_values('observation_date')
    
    # Create time series
    ts = pd.Series(
        data=ind_data['value_numeric'].values,
        index=pd.DatetimeIndex(ind_data['observation_date'])
    )
    
    # Resample if needed
    if freq == 'A':  # Annual - take last value per year
        ts = ts.resample('Y').last()
    elif freq == 'M':  # Monthly - interpolate
        # Create monthly index
        monthly_idx = pd.date_range(start=ts.index.min(), end=ts.index.max(), freq='M')
        ts = ts.reindex(monthly_idx)
        ts = ts.interpolate(method='linear')
    
    # Metadata
    metadata = {
        'n_points': len(ind_data),
        'date_range': (ts.index.min(), ts.index.max()),
        'min_value': ts.min(),
        'max_value': ts.max(),
        'trend': 'increasing' if ts.iloc[-1] > ts.iloc[0] else 'decreasing',
        'growth_rate': ((ts.iloc[-1] / ts.iloc[0]) ** (1/(len(ts)-1)) - 1) * 100 if len(ts) > 1 else None
    }
    
    return ts, metadata

# %%
# Prepare time series for primary targets
print("\n📈 Preparing Time Series Data:")

time_series = {}
metadata = {}

for target_code in primary_targets.keys():
    ts, meta = prepare_time_series(target_code, freq='A')
    if ts is not None:
        time_series[target_code] = ts
        metadata[target_code] = meta
        print(f"✓ {primary_targets[target_code]['name']}: {len(ts)} points from {meta['date_range'][0].year} to {meta['date_range'][1].year}")
        print(f"  Growth rate: {meta['growth_rate']:.2f}% per year")
        print(f"  Values: {list(ts.values.round(1))}")

# Also prepare enabling indicators
for indicator in enabling_indicators:
    ts, meta = prepare_time_series(indicator, freq='A')
    if ts is not None:
        time_series[indicator] = ts
        metadata[indicator] = meta


### 5. Baseline Model: Trend Regression

In [ ]:
def fit_trend_model(ts, model_type='linear', forecast_years=3):
    """
    Fit a trend model to time series data.
    
    Parameters:
    -----------
    ts : pd.Series
        Time series data
    model_type : str
        Type of model: 'linear', 'log', 'exp'
    forecast_years : int
        Number of years to forecast ahead
    
    Returns:
    --------
    forecast : pd.Series
        Forecasted values
    model_params : dict
        Model parameters
    ci : tuple
        Confidence intervals (lower, upper)
    """
    
    # Convert time to numeric (years since start)
    time_idx = np.array([(d.year - ts.index[0].year) for d in ts.index])
    values = ts.values
    
    if model_type == 'linear':
        # Linear regression: y = a + b*t
        X = sm.add_constant(time_idx)
        model = sm.OLS(values, X).fit()
        
        # Forecast
        future_time = np.arange(time_idx[-1] + 1, time_idx[-1] + forecast_years + 1)
        X_future = sm.add_constant(future_time)
        forecast_vals = model.predict(X_future)
        
        # Confidence intervals
        predictions = model.get_prediction(X_future)
        ci = predictions.conf_int(alpha=0.2)  # 80% CI
        
    elif model_type == 'log':
        # Log-linear: ln(y) = a + b*t
        X = sm.add_constant(time_idx)
        y_log = np.log(values)
        model = sm.OLS(y_log, X).fit()
        
        # Forecast
        future_time = np.arange(time_idx[-1] + 1, time_idx[-1] + forecast_years + 1)
        X_future = sm.add_constant(future_time)
        forecast_log = model.predict(X_future)
        forecast_vals = np.exp(forecast_log)
        
        # Confidence intervals (approximate)
        se = model.bse[1]  # Standard error of slope
        ci_factor = 1.28 * se * forecast_vals  # 80% CI approximation
        ci_lower = forecast_vals - ci_factor
        ci_upper = forecast_vals + ci_factor
        ci = (ci_lower, ci_upper)
        
    elif model_type == 'exp':
        # Exponential: y = a * exp(b*t)
        def exp_func(t, a, b):
            return a * np.exp(b * t)
        
        # Fit using curve_fit
        popt, pcov = curve_fit(exp_func, time_idx, values, p0=[values[0], 0.1])
        
        # Forecast
        future_time = np.arange(time_idx[-1] + 1, time_idx[-1] + forecast_years + 1)
        forecast_vals = exp_func(future_time, *popt)
        
        # Confidence intervals (approximate)
        perr = np.sqrt(np.diag(pcov))
        ci_factor = 1.28 * perr[1] * forecast_vals
        ci_lower = forecast_vals - ci_factor
        ci_upper = forecast_vals + ci_factor
        ci = (ci_lower, ci_upper)
        
        model = {'params': popt, 'covariance': pcov}
    
    # Create forecast series
    forecast_dates = pd.date_range(
        start=ts.index[-1] + pd.DateOffset(years=1),
        periods=forecast_years,
        freq='Y'
    )
    
    forecast_series = pd.Series(forecast_vals, index=forecast_dates)
    
    # Model parameters
    model_params = {
        'type': model_type,
        'r_squared': model.rsquared if hasattr(model, 'rsquared') else None,
        'growth_rate': None
    }
    
    if model_type == 'linear' and hasattr(model, 'params'):
        model_params['intercept'] = model.params[0]
        model_params['slope'] = model.params[1]
        model_params['growth_rate'] = model.params[1] / ts.values[-1] * 100
    
    return forecast_series, model_params, ci

# %%
print("\n" + "="*60)
print("BASELINE FORECASTS: TREND CONTINUATION")
print("="*60)

baseline_forecasts = {}
baseline_models = {}

for target_code in primary_targets.keys():
    if target_code in time_series:
        ts = time_series[target_code]
        
        print(f"\n📊 {primary_targets[target_code]['name']}:")
        print(f"  Historical data: {len(ts)} points ({ts.index[0].year}-{ts.index[-1].year})")
        
        # Try different trend models
        models_to_try = ['linear', 'log']  # Skip exponential for now
        
        best_forecast = None
        best_model = None
        best_ci = None
        best_score = -np.inf
        
        for model_type in models_to_try:
            try:
                forecast, model_params, ci = fit_trend_model(ts, model_type=model_type, forecast_years=3)
                
                # Simple model selection: higher R² is better
                score = model_params.get('r_squared', 0) if model_params.get('r_squared') is not None else 0
                
                if score > best_score:
                    best_score = score
                    best_forecast = forecast
                    best_model = model_params
                    best_ci = ci
                    
                print(f"  {model_type.capitalize()} model: R² = {score:.3f}")
                for i, (date, val) in enumerate(forecast.items(), 1):
                    print(f"    202{4+i}: {val:.1f}%")
                    
            except Exception as e:
                print(f"  {model_type.capitalize()} model failed: {str(e)[:50]}...")
        
        # Store best model
        if best_forecast is not None:
            baseline_forecasts[target_code] = best_forecast
            baseline_models[target_code] = best_model
            
            # Add confidence intervals
            ci_lower, ci_upper = best_ci
            baseline_forecasts[target_code + '_lower'] = pd.Series(ci_lower, index=best_forecast.index)
            baseline_forecasts[target_code + '_upper'] = pd.Series(ci_upper, index=best_forecast.index)


### 6. Event-Augmented Forecasting Model

In [ ]:
print("\n" + "="*60)
print("EVENT-AUGMENTED FORECASTING MODEL")
print("="*60)

# Define future events (2025-2027)
future_events = [
    # Already happened but effects continue
    ('Telebirr launch', pd.Timestamp('2021-05-01')),
    ('M-Pesa entry', pd.Timestamp('2023-08-01')),
    ('Interoperability regulation enacted', pd.Timestamp('2024-07-01')),
    ('Fayda Digital ID mass enrollment', pd.Timestamp('2024-10-15')),
    
    # Planned/anticipated events
    ('National QR code merchant rollout', pd.Timestamp('2025-01-01')),
    ('Mobile money tax incentives', pd.Timestamp('2025-06-01')),  # Hypothetical
    ('Rural agent network expansion', pd.Timestamp('2026-01-01')),  # Hypothetical
    ('Digital literacy campaign', pd.Timestamp('2026-07-01'))  # Hypothetical
]

print("\n📅 Events Incorporated in Model:")
for event_name, event_date in future_events:
    print(f"  • {event_name}: {event_date.strftime('%b %Y')}")

# %%
def calculate_event_impact_adjustment(base_forecast, events, indicator, impact_matrix, lag_matrix):
    """
    Calculate adjustment to baseline forecast based on event impacts.
    
    Parameters:
    -----------
    base_forecast : pd.Series
        Baseline forecast values
    events : list of tuples
        List of (event_name, event_date)
    indicator : str
        Indicator code
    impact_matrix : pd.DataFrame
        Event-indicator impact matrix
    lag_matrix : pd.DataFrame
        Event lag matrix
    
    Returns:
    --------
    adjustment_series : pd.Series
        Adjustment values (in percentage points)
    impact_details : dict
        Detailed impact breakdown
    """
    
    adjustment_series = pd.Series(0.0, index=base_forecast.index)
    impact_details = {}
    
    for event_name, event_date in events:
        # Skip if event is after forecast period
        if event_date > base_forecast.index[-1]:
            continue
            
        # Get impact score from matrix
        if event_name in impact_matrix.index and indicator in impact_matrix.columns:
            impact_score = impact_matrix.loc[event_name, indicator]
            
            # Skip if no impact
            if pd.isna(impact_score) or impact_score == 0:
                continue
                
            # Get lag
            lag = lag_matrix.loc[event_name, indicator] if event_name in lag_matrix.index and indicator in lag_matrix.columns else 6
            if pd.isna(lag):
                lag = 6  # Default 6 months
                
            # Calculate impact for each forecast date
            event_impacts = {}
            for forecast_date in base_forecast.index:
                # Calculate months since event
                months_since = (forecast_date.year - event_date.year) * 12 + (forecast_date.month - event_date.month)
                
                if months_since < 0:
                    # Event hasn't happened yet at forecast date
                    impact = 0
                elif months_since < lag:
                    # Impact building up
                    buildup_factor = months_since / lag
                    impact = impact_score * buildup_factor
                else:
                    # Full impact
                    impact = impact_score
                    
                # Convert impact score to percentage points
                # Impact score of 1.0 = 10 percentage points (based on calibration)
                impact_pp = impact * 10
                
                # Store
                event_impacts[forecast_date] = impact_pp
                adjustment_series[forecast_date] += impact_pp
            
            impact_details[event_name] = {
                'impact_score': impact_score,
                'lag_months': lag,
                'annual_impacts': event_impacts
            }
    
    return adjustment_series, impact_details

# %%
print("\n🎯 Generating Event-Augmented Forecasts:")

event_augmented_forecasts = {}
event_impact_details = {}

for target_code in primary_targets.keys():
    if target_code in baseline_forecasts:
        base_forecast = baseline_forecasts[target_code]
        
        print(f"\n📊 {primary_targets[target_code]['name']}:")
        print(f"  Baseline forecast: {base_forecast.values.round(1)}%")
        
        # Calculate event impacts
        adjustment, details = calculate_event_impact_adjustment(
            base_forecast, future_events, target_code, 
            association_matrix, lag_matrix
        )
        
        # Apply adjustment
        adjusted_forecast = base_forecast + adjustment
        event_augmented_forecasts[target_code] = adjusted_forecast
        event_impact_details[target_code] = details
        
        print(f"  Event adjustment: {adjustment.values.round(1)}pp")
        print(f"  Adjusted forecast: {adjusted_forecast.values.round(1)}%")
        
        # Show top contributing events
        if details:
            print(f"  Top event impacts for 2027:")
            event_contributions = {}
            for event_name, event_details in details.items():
                contribution = event_details['annual_impacts'].get(adjusted_forecast.index[-1], 0)
                if abs(contribution) > 0.1:  # Only show significant contributions
                    event_contributions[event_name] = contribution
            
            # Sort by absolute contribution
            for event_name, contribution in sorted(event_contributions.items(), key=lambda x: abs(x[1]), reverse=True)[:3]:
                print(f"    • {event_name}: {contribution:+.1f}pp")

### 7. Scenario Analysis

In [ ]:
print("\n" + "="*60)
print("SCENARIO ANALYSIS")
print("="*60)

# Define scenarios
scenarios = {
    'optimistic': {
        'description': 'Accelerated digital transformation, favorable policies, strong economic growth',
        'growth_multiplier': 1.3,  # 30% higher growth
        'event_impact_multiplier': 1.2,  # 20% stronger event impacts
        'probability': 0.25  # 25% probability
    },
    'base': {
        'description': 'Continuation of current trends, planned interventions proceed as expected',
        'growth_multiplier': 1.0,
        'event_impact_multiplier': 1.0,
        'probability': 0.50  # 50% probability
    },
    'pessimistic': {
        'description': 'Economic challenges, implementation delays, slower adoption',
        'growth_multiplier': 0.7,  # 30% lower growth
        'event_impact_multiplier': 0.8,  # 20% weaker event impacts
        'probability': 0.25  # 25% probability
    }
}

print("\n📊 Defined Scenarios:")
for scenario_name, params in scenarios.items():
    print(f"\n{scenario_name.upper()} Scenario:")
    print(f"  {params['description']}")
    print(f"  Growth multiplier: {params['growth_multiplier']}")
    print(f"  Event impact multiplier: {params['event_impact_multiplier']}")
    print(f"  Probability: {params['probability']*100:.0f}%")

# %%
def generate_scenario_forecasts(base_forecast, event_adjustment, scenario_params):
    """
    Generate forecasts for a specific scenario.
    
    Parameters:
    -----------
    base_forecast : pd.Series
        Baseline trend forecast
    event_adjustment : pd.Series
        Event impact adjustment
    scenario_params : dict
        Scenario parameters
    
    Returns:
    --------
    scenario_forecast : pd.Series
        Scenario-specific forecast
    """
    
    # Apply multipliers
    adjusted_trend = base_forecast.copy()
    
    # Adjust trend based on multiplier
    # Calculate incremental growth from baseline
    if len(adjusted_trend) > 1:
        # Get the growth from the last historical point
        last_historical = base_forecast.index[0] - pd.DateOffset(years=1)
        # This is simplified - in practice, we'd adjust the trend model parameters
        pass
    
    # Simpler approach: Apply multiplier to the difference from current value
    current_value = primary_targets[base_forecast.name]['current_value']
    if current_value:
        for date in adjusted_trend.index:
            base_increment = base_forecast[date] - current_value
            adjusted_increment = base_increment * scenario_params['growth_multiplier']
            adjusted_trend[date] = current_value + adjusted_increment
    
    # Adjust event impacts
    adjusted_events = event_adjustment * scenario_params['event_impact_multiplier']
    
    # Combine
    scenario_forecast = adjusted_trend + adjusted_events
    
    return scenario_forecast

# %%
print("\n🎯 Generating Scenario Forecasts:")

scenario_forecasts = {scenario: {} for scenario in scenarios.keys()}

for target_code in primary_targets.keys():
    if target_code in baseline_forecasts and target_code in event_augmented_forecasts:
        base_forecast = baseline_forecasts[target_code]
        event_adjustment = event_augmented_forecasts[target_code] - base_forecast
        
        print(f"\n📊 {primary_targets[target_code]['name']}:")
        
        for scenario_name, scenario_params in scenarios.items():
            scenario_forecast = generate_scenario_forecasts(
                base_forecast.copy(),
                event_adjustment.copy(),
                scenario_params
            )
            
            scenario_forecasts[scenario_name][target_code] = scenario_forecast
            
            print(f"  {scenario_name.capitalize()} scenario 2027: {scenario_forecast.iloc[-1]:.1f}%")

### 8. Quantify Uncertainty

In [ ]:
print("\n" + "="*60)
print("UNCERTAINTY QUANTIFICATION")
print("="*60)

# Multiple sources of uncertainty
uncertainty_sources = {
    'data_sparsity': {
        'description': 'Limited historical data points (only 5 Findex surveys)',
        'impact': 'High',
        'quantification': 'Wider confidence intervals (±3-5pp)'
    },
    'event_impact_estimation': {
        'description': 'Estimated impacts of future events',
        'impact': 'Medium-High',
        'quantification': 'Scenario analysis, sensitivity testing'
    },
    'model_selection': {
        'description': 'Choice of forecasting model',
        'impact': 'Medium',
        'quantification': 'Multiple model comparison'
    },
    'external_factors': {
        'description': 'Economic conditions, policy changes, technological disruptions',
        'impact': 'High',
        'quantification': 'Scenario analysis'
    }
}

print("\n📊 Sources of Uncertainty:")
for source, details in uncertainty_sources.items():
    print(f"\n• {source.replace('_', ' ').title()}:")
    print(f"  {details['description']}")
    print(f"  Impact: {details['impact']}")
    print(f"  Quantification: {details['quantification']}")

# %%
def calculate_confidence_intervals(forecast_series, confidence_level=0.8):
    """
    Calculate confidence intervals for forecasts.
    
    Parameters:
    -----------
    forecast_series : pd.Series
        Forecast values
    confidence_level : float
        Confidence level (0.8 for 80%, 0.95 for 95%)
    
    Returns:
    --------
    ci_lower : pd.Series
        Lower bound of confidence interval
    ci_upper : pd.Series
        Upper bound of confidence interval
    """
    
    # Z-score for confidence level
    z_score = stats.norm.ppf(1 - (1 - confidence_level) / 2)
    
    # Estimate uncertainty based on:
    # 1. Historical forecast errors (if available)
    # 2. Expert judgment for different horizons
    
    # For now, use simple rule based on forecast horizon
    ci_lower = pd.Series(index=forecast_series.index, dtype=float)
    ci_upper = pd.Series(index=forecast_series.index, dtype=float)
    
    for i, (date, value) in enumerate(forecast_series.items(), 1):
        # Uncertainty increases with forecast horizon
        uncertainty_factor = 0.5 + (i * 0.3)  # 0.8pp for year 1, 1.1pp for year 2, 1.4pp for year 3
        
        margin = uncertainty_factor * z_score
        
        ci_lower[date] = max(0, value - margin)  # Don't go below 0%
        ci_upper[date] = min(100, value + margin)  # Don't go above 100%
    
    return ci_lower, ci_upper

# %%
print("\n🎯 Calculating Confidence Intervals:")

forecast_with_uncertainty = {}

for target_code in primary_targets.keys():
    if target_code in event_augmented_forecasts:
        forecast = event_augmented_forecasts[target_code]
        
        # Calculate confidence intervals
        ci_80_lower, ci_80_upper = calculate_confidence_intervals(forecast, confidence_level=0.8)
        ci_95_lower, ci_95_upper = calculate_confidence_intervals(forecast, confidence_level=0.95)
        
        forecast_with_uncertainty[target_code] = {
            'forecast': forecast,
            'ci_80_lower': ci_80_lower,
            'ci_80_upper': ci_80_upper,
            'ci_95_lower': ci_95_lower,
            'ci_95_upper': ci_95_upper
        }
        
        print(f"\n📊 {primary_targets[target_code]['name']}:")
        for date in forecast.index:
            val = forecast[date]
            lower_80 = ci_80_lower[date]
            upper_80 = ci_80_upper[date]
            
            print(f"  {date.year}: {val:.1f}% (80% CI: {lower_80:.1f}% - {upper_80:.1f}%)")


### 9. Visualize Forecasts

In [ ]:
print("\n" + "="*60)
print("FORECAST VISUALIZATION")
print("="*60)

# Create comprehensive forecast visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Colors for scenarios
scenario_colors = {
    'optimistic': 'green',
    'base': 'blue',
    'pessimistic': 'red'
}

# 1. Account Ownership Forecast
ax1 = axes[0, 0]
target_code = 'ACC_OWNERSHIP'

if target_code in time_series and target_code in forecast_with_uncertainty:
    # Historical data
    historical = time_series[target_code]
    ax1.plot(historical.index, historical.values, 'ko-', linewidth=2, markersize=8, label='Historical')
    
    # Forecast
    forecast_data = forecast_with_uncertainty[target_code]
    forecast_series = forecast_data['forecast']
    
    # Plot forecast with confidence intervals
    ax1.plot(forecast_series.index, forecast_series.values, 'b-o', linewidth=2, markersize=8, label='Base Forecast')
    
    # 80% confidence interval
    ax1.fill_between(forecast_series.index, 
                     forecast_data['ci_80_lower'], 
                     forecast_data['ci_80_upper'], 
                     alpha=0.2, color='blue', label='80% CI')
    
    # Scenarios
    for scenario_name in scenarios.keys():
        if scenario_name in scenario_forecasts and target_code in scenario_forecasts[scenario_name]:
            scenario_vals = scenario_forecasts[scenario_name][target_code]
            ax1.plot(scenario_vals.index, scenario_vals.values, '--', 
                     color=scenario_colors[scenario_name], alpha=0.7, 
                     label=f'{scenario_name.capitalize()} Scenario')
    
    # NFIS target
    nfis_target = primary_targets[target_code]['nfis_target']
    if nfis_target:
        ax1.axhline(y=nfis_target, color='red', linestyle=':', linewidth=2, alpha=0.7, label=f'NFIS Target ({nfis_target}%)')
    
    ax1.set_title('Account Ownership Forecast (2025-2027)', fontweight='bold', fontsize=12)
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Account Ownership (%)')
    ax1.legend(loc='upper left', fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # Add value annotations
    for date, val in forecast_series.items():
        ax1.text(date, val + 0.5, f'{val:.1f}%', ha='center', fontweight='bold')

# 2. Digital Payment Forecast
ax2 = axes[0, 1]
target_code = 'USG_DIGITAL_PAYMENT'

if target_code in time_series and target_code in forecast_with_uncertainty:
    # Historical data
    historical = time_series[target_code]
    ax2.plot(historical.index, historical.values, 'ko-', linewidth=2, markersize=8, label='Historical')
    
    # Forecast
    forecast_data = forecast_with_uncertainty[target_code]
    forecast_series = forecast_data['forecast']
    
    # Plot forecast with confidence intervals
    ax2.plot(forecast_series.index, forecast_series.values, 'g-o', linewidth=2, markersize=8, label='Base Forecast')
    
    # 80% confidence interval
    ax2.fill_between(forecast_series.index, 
                     forecast_data['ci_80_lower'], 
                     forecast_data['ci_80_upper'], 
                     alpha=0.2, color='green', label='80% CI')
    
    # Scenarios
    for scenario_name in scenarios.keys():
        if scenario_name in scenario_forecasts and target_code in scenario_forecasts[scenario_name]:
            scenario_vals = scenario_forecasts[scenario_name][target_code]
            ax2.plot(scenario_vals.index, scenario_vals.values, '--', 
                     color=scenario_colors[scenario_name], alpha=0.7, 
                     label=f'{scenario_name.capitalize()} Scenario')
    
    ax2.set_title('Digital Payment Adoption Forecast (2025-2027)', fontweight='bold', fontsize=12)
    ax2.set_xlabel('Year')
    ax2.set_ylabel('Digital Payment Adoption (%)')
    ax2.legend(loc='upper left', fontsize=9)
    ax2.grid(True, alpha=0.3)
    
    # Add value annotations
    for date, val in forecast_series.items():
        ax2.text(date, val + 0.5, f'{val:.1f}%', ha='center', fontweight='bold')

# 3. Scenario Comparison (2027 values)
ax3 = axes[1, 0]
scenario_data = []
labels = []

for target_code in primary_targets.keys():
    if target_code in forecast_with_uncertainty:
        label = primary_targets[target_code]['name']
        labels.append(label)
        
        scenario_values = []
        for scenario_name in ['pessimistic', 'base', 'optimistic']:
            if scenario_name in scenario_forecasts and target_code in scenario_forecasts[scenario_name]:
                scenario_values.append(scenario_forecasts[scenario_name][target_code].iloc[-1])
        
        scenario_data.append(scenario_values)

if scenario_data:
    x = np.arange(len(labels))
    width = 0.25
    
    ax3.bar(x - width, [vals[0] for vals in scenario_data], width, label='Pessimistic', color='red', alpha=0.7)
    ax3.bar(x, [vals[1] for vals in scenario_data], width, label='Base', color='blue', alpha=0.7)
    ax3.bar(x + width, [vals[2] for vals in scenario_data], width, label='Optimistic', color='green', alpha=0.7)
    
    ax3.set_title('Scenario Comparison: 2027 Forecasts', fontweight='bold', fontsize=12)
    ax3.set_xlabel('Indicator')
    ax3.set_ylabel('Value (%)')
    ax3.set_xticks(x)
    ax3.set_xticklabels([l[:20] + '...' if len(l) > 20 else l for l in labels], rotation=45, ha='right')
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for i, (pess, base, opt) in enumerate(scenario_data):
        ax3.text(i - width, pess + 0.5, f'{pess:.1f}', ha='center', fontsize=8, fontweight='bold')
        ax3.text(i, base + 0.5, f'{base:.1f}', ha='center', fontsize=8, fontweight='bold')
        ax3.text(i + width, opt + 0.5, f'{opt:.1f}', ha='center', fontsize=8, fontweight='bold')

# 4. Progress Toward NFIS Target
ax4 = axes[1, 1]
target_code = 'ACC_OWNERSHIP'

if target_code in forecast_with_uncertainty and primary_targets[target_code]['nfis_target']:
    nfis_target = primary_targets[target_code]['nfis_target']
    current_value = primary_targets[target_code]['current_value']
    
    if current_value:
        # Data for progress chart
        years = ['2024', '2025', '2026', '2027']
        base_values = [current_value] + list(event_augmented_forecasts[target_code].values)
        optimistic_values = [current_value] + list(scenario_forecasts['optimistic'][target_code].values)
        pessimistic_values = [current_value] + list(scenario_forecasts['pessimistic'][target_code].values)
        
        x_pos = np.arange(len(years))
        
        ax4.plot(x_pos, base_values, 'b-o', linewidth=2, markersize=8, label='Base Forecast')
        ax4.plot(x_pos, optimistic_values, 'g--o', linewidth=2, markersize=6, label='Optimistic')
        ax4.plot(x_pos, pessimistic_values, 'r--o', linewidth=2, markersize=6, label='Pessimistic')
        ax4.axhline(y=nfis_target, color='red', linestyle=':', linewidth=2, alpha=0.7, label=f'NFIS Target ({nfis_target}%)')
        
        ax4.set_title('Progress Toward NFIS Target (60% by 2027)', fontweight='bold', fontsize=12)
        ax4.set_xlabel('Year')
        ax4.set_ylabel('Account Ownership (%)')
        ax4.set_xticks(x_pos)
        ax4.set_xticklabels(years)
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        # Add value labels
        for i, val in enumerate(base_values):
            ax4.text(i, val + 0.5, f'{val:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../data/processed/forecast_visualizations.png', dpi=300, bbox_inches='tight')
plt.show()

### 10. Interactive Forecast Visualization

In [ ]:
# Create interactive plot with Plotly
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Account Ownership Forecast', 'Digital Payment Forecast', 
                    'Scenario Comparison (2027)', 'Progress to NFIS Target'),
    vertical_spacing=0.15,
    horizontal_spacing=0.15
)

# 1. Account Ownership Forecast
target_code = 'ACC_OWNERSHIP'
if target_code in time_series and target_code in forecast_with_uncertainty:
    # Historical
    historical = time_series[target_code]
    fig.add_trace(
        go.Scatter(x=historical.index, y=historical.values,
                   mode='lines+markers',
                   name='Historical',
                   line=dict(color='black', width=2),
                   marker=dict(size=8)),
        row=1, col=1
    )
    
    # Forecast
    forecast_data = forecast_with_uncertainty[target_code]
    forecast_series = forecast_data['forecast']
    
    fig.add_trace(
        go.Scatter(x=forecast_series.index, y=forecast_series.values,
                   mode='lines+markers',
                   name='Base Forecast',
                   line=dict(color='blue', width=2),
                   marker=dict(size=8)),
        row=1, col=1
    )
    
    # Confidence interval
    fig.add_trace(
        go.Scatter(x=forecast_series.index.tolist() + forecast_series.index.tolist()[::-1],
                   y=forecast_data['ci_80_upper'].tolist() + forecast_data['ci_80_lower'].tolist()[::-1],
                   fill='toself',
                   fillcolor='rgba(0, 100, 255, 0.2)',
                   line=dict(color='rgba(255,255,255,0)'),
                   name='80% Confidence Interval'),
        row=1, col=1
    )

# 2. Digital Payment Forecast
target_code = 'USG_DIGITAL_PAYMENT'
if target_code in time_series and target_code in forecast_with_uncertainty:
    # Historical
    historical = time_series[target_code]
    fig.add_trace(
        go.Scatter(x=historical.index, y=historical.values,
                   mode='lines+markers',
                   name='Historical',
                   line=dict(color='black', width=2),
                   marker=dict(size=8),
                   showlegend=False),
        row=1, col=2
    )
    
    # Forecast
    forecast_data = forecast_with_uncertainty[target_code]
    forecast_series = forecast_data['forecast']
    
    fig.add_trace(
        go.Scatter(x=forecast_series.index, y=forecast_series.values,
                   mode='lines+markers',
                   name='Base Forecast',
                   line=dict(color='green', width=2),
                   marker=dict(size=8),
                   showlegend=False),
        row=1, col=2
    )
    
    # Confidence interval
    fig.add_trace(
        go.Scatter(x=forecast_series.index.tolist() + forecast_series.index.tolist()[::-1],
                   y=forecast_data['ci_80_upper'].tolist() + forecast_data['ci_80_lower'].tolist()[::-1],
                   fill='toself',
                   fillcolor='rgba(0, 255, 100, 0.2)',
                   line=dict(color='rgba(255,255,255,0)'),
                   name='80% Confidence Interval',
                   showlegend=False),
        row=1, col=2
    )

# Update layout
fig.update_layout(height=800, width=1200, 
                  title_text="Ethiopia Financial Inclusion Forecasts 2025-2027",
                  showlegend=True)

# Save interactive plot
fig.write_html('../data/processed/interactive_forecasts.html')
print("✓ Interactive forecast visualization saved")


### 11. Forecast Interpretation and Insights

In [ ]:
print("\n" + "="*60)
print("FORECAST INTERPRETATION AND INSIGHTS")
print("="*60)

# Generate key insights from forecasts
forecast_insights = []

# Insight 1: Account Ownership trajectory
target_code = 'ACC_OWNERSHIP'
if target_code in event_augmented_forecasts:
    forecast_2027 = event_augmented_forecasts[target_code].iloc[-1]
    current = primary_targets[target_code]['current_value']
    nfis_target = primary_targets[target_code]['nfits_target']
    
    insight = {
        'title': '📈 Account Ownership Growth Trajectory',
        'description': f'Account ownership is forecast to reach {forecast_2027:.1f}% by 2027, growing from {current}% in 2024.',
        'implication': f'This represents {forecast_2027 - current:.1f}pp growth over 3 years.'
    }
    if nfis_target:
        gap = nfis_target - forecast_2027
        insight['description'] += f' The NFIS target of {nfis_target}% would require an additional {gap:.1f}pp growth.'
        insight['implication'] += f' To reach the NFIS target, growth needs to accelerate to ~{gap/3:.1f}pp per year.'
    forecast_insights.append(insight)

# Insight 2: Digital Payment growth rate
target_code = 'USG_DIGITAL_PAYMENT'
if target_code in event_augmented_forecasts:
    forecast_2027 = event_augmented_forecasts[target_code].iloc[-1]
    current = primary_targets[target_code]['current_value']
    annual_growth = (forecast_2027 - current) / 3
    
    insight = {
        'title': '🚀 Digital Payments Accelerating',
        'description': f'Digital payment adoption forecast to grow at {annual_growth:.1f}pp per year, reaching {forecast_2027:.1f}% by 2027.',
        'implication': 'Digital payments growing faster than account ownership, indicating usage deepening.'
    }
    forecast_insights.append(insight)

# Insight 3: Scenario ranges
if 'ACC_OWNERSHIP' in scenario_forecasts['optimistic']:
    optimistic_2027 = scenario_forecasts['optimistic']['ACC_OWNERSHIP'].iloc[-1]
    pessimistic_2027 = scenario_forecasts['pessimistic']['ACC_OWNERSHIP'].iloc[-1]
    range_2027 = optimistic_2027 - pessimistic_2027
    
    insight = {
        'title': '📊 Significant Uncertainty Range',
        'description': f'Scenario analysis shows a {range_2027:.1f}pp range for 2027 account ownership ({pessimistic_2027:.1f}% to {optimistic_2027:.1f}%).',
        'implication': 'Policy interventions and economic conditions can significantly impact outcomes.'
    }
    forecast_insights.append(insight)

# Insight 4: Key drivers
if event_impact_details and 'ACC_OWNERSHIP' in event_impact_details:
    impacts = event_impact_details['ACC_OWNERSHIP']
    if impacts:
        # Find event with largest impact in 2027
        max_impact = 0
        max_event = None
        
        for event_name, details in impacts.items():
            impact_2027 = details['annual_impacts'].get(event_augmented_forecasts['ACC_OWNERSHIP'].index[-1], 0)
            if abs(impact_2027) > abs(max_impact):
                max_impact = impact_2027
                max_event = event_name
        
        if max_event:
            insight = {
                'title': '🎯 Key Driver: Interoperability',
                'description': f'The {max_event} is forecast to contribute {max_impact:+.1f}pp to account ownership by 2027.',
                'implication': 'Interoperability policies have the largest potential impact on inclusion.'
            }
            forecast_insights.append(insight)

# Insight 5: Usage vs Access gap
if 'ACC_OWNERSHIP' in event_augmented_forecasts and 'USG_DIGITAL_PAYMENT' in event_augmented_forecasts:
    acc_2027 = event_augmented_forecasts['ACC_OWNERSHIP'].iloc[-1]
    usage_2027 = event_augmented_forecasts['USG_DIGITAL_PAYMENT'].iloc[-1]
    gap_2027 = acc_2027 - usage_2027
    
    insight = {
        'title': '⚡ Usage Gap Persists',
        'description': f'The gap between account ownership ({acc_2027:.1f}%) and active usage ({usage_2027:.1f}%) is forecast to be {gap_2027:.1f}pp in 2027.',
        'implication': 'Need to focus on activating existing accounts, not just opening new ones.'
    }
    forecast_insights.append(insight)

print("\n🔍 Key Forecast Insights:")
for i, insight in enumerate(forecast_insights, 1):
    print(f"\n{i}. {insight['title']}")
    print(f"   {insight['description']}")
    print(f"   Implication: {insight['implication']}")

# Save insights
forecast_insights_df = pd.DataFrame(forecast_insights)
forecast_insights_df.to_csv('../data/processed/forecast_insights.csv', index=False)


### 12. Forecast Tables for Reporting

In [ ]:
print("\n" + "="*60)
print("FORECAST TABLES")
print("="*60)

# Create comprehensive forecast table
forecast_tables = {}

for target_code in primary_targets.keys():
    if target_code in event_augmented_forecasts:
        forecast_data = forecast_with_uncertainty[target_code]
        forecast_series = forecast_data['forecast']
        
        table_data = []
        for date in forecast_series.index:
            row = {
                'Year': date.year,
                'Base Forecast (%)': forecast_series[date],
                '80% CI Lower (%)': forecast_data['ci_80_lower'][date],
                '80% CI Upper (%)': forecast_data['ci_80_upper'][date],
                '95% CI Lower (%)': forecast_data['ci_95_lower'][date],
                '95% CI Upper (%)': forecast_data['ci_95_upper'][date]
            }
            
            # Add scenario values
            for scenario_name in scenarios.keys():
                if scenario_name in scenario_forecasts and target_code in scenario_forecasts[scenario_name]:
                    row[f'{scenario_name.capitalize()} Scenario (%)'] = scenario_forecasts[scenario_name][target_code][date]
            
            table_data.append(row)
        
        forecast_table = pd.DataFrame(table_data)
        forecast_tables[target_code] = forecast_table
        
        print(f"\n📋 {primary_targets[target_code]['name']} Forecast Table:")
        display(forecast_table.round(1))

# Save forecast tables
for target_code, table in forecast_tables.items():
    filename = f'../data/processed/forecast_table_{target_code}.csv'
    table.to_csv(filename, index=False)
    print(f"✓ Saved: {filename}")

### 13. Limitations and Recommendations

In [ ]:
print("\n" + "="*60)
print("FORECAST LIMITATIONS AND RECOMMENDATIONS")
print("="*60)

forecast_limitations = [
    {
        'limitation': 'Sparse historical data',
        'impact': 'High uncertainty in trend estimation',
        'recommendation': 'Supplement with high-frequency proxy data (transaction volumes, agent registrations)'
    },
    {
        'limitation': 'Event impact estimation uncertainty',
        'impact': 'Potential over/under-estimation of policy effects',
        'recommendation': 'Regularly update impact estimates as new data becomes available'
    },
    {
        'limitation': 'Assumption of trend continuation',
        'impact': 'Misses potential structural breaks or accelerations',
        'recommendation': 'Monitor leading indicators (infrastructure, digital ID enrollments)'
    },
    {
        'limitation': 'Limited economic variables',
        'impact': 'Does not account for economic shocks or inflation effects',
        'recommendation': 'Incorporate GDP growth, inflation, and employment data'
    },
    {
        'limitation': 'National-level focus',
        'impact': 'Misses regional variations and urban-rural divides',
        'recommendation': 'Develop regional forecasts when disaggregated data becomes available'
    }
]

print("\n⚠️  Forecast Limitations:")
for i, limit in enumerate(forecast_limitations, 1):
    print(f"\n{i}. {limit['limitation']}")
    print(f"   Impact: {limit['impact']}")
    print(f"   Recommendation: {limit['recommendation']}")

# Save limitations
forecast_limitations_df = pd.DataFrame(forecast_limitations)
forecast_limitations_df.to_csv('../data/processed/forecast_limitations.csv', index=False)


### 14. Task 4 Completion Summary

In [ ]:
print("\n" + "="*60)
print("TASK 4 COMPLETION SUMMARY")
print("="*60)

# Summary statistics
summary_stats = {
    'targets_forecasted': len(primary_targets),
    'forecast_horizon': '2025-2027 (3 years)',
    'scenarios_developed': len(scenarios),
    'confidence_intervals': '80% and 95%',
    'key_insights_generated': len(forecast_insights)
}

print("\n📊 Forecast Summary:")
for key, value in summary_stats.items():
    print(f"  {key.replace('_', ' ').title()}: {value}")

print("\n🎯 Key Forecast Results:")
for target_code in primary_targets.keys():
    if target_code in event_augmented_forecasts:
        forecast_2027 = event_augmented_forecasts[target_code].iloc[-1]
        current = primary_targets[target_code]['current_value']
        growth = forecast_2027 - current
        
        print(f"\n  {primary_targets[target_code]['name']}:")
        print(f"    Current (2024): {current}%")
        print(f"    Forecast 2027: {forecast_2027:.1f}%")
        print(f"    Growth (2024-27): {growth:+.1f}pp")
        
        if target_code == 'ACC_OWNERSHIP' and primary_targets[target_code]['nfis_target']:
            nfis_gap = primary_targets[target_code]['nfis_target'] - forecast_2027
            print(f"    Gap to NFIS target: {nfis_gap:.1f}pp")

print("\n📁 Output Files Generated:")
output_files = [
    '../data/processed/forecast_visualizations.png',
    '../data/processed/interactive_forecasts.html',
    '../data/processed/forecast_insights.csv',
    '../data/processed/forecast_limitations.csv',
    '../data/processed/forecast_table_ACC_OWNERSHIP.csv',
    '../data/processed/forecast_table_USG_DIGITAL_PAYMENT.csv'
]

for file in output_files:
    print(f"  ✓ {file}"